# Minimal Blockchain in Python

Creating a minimal private blockchain with essential operations such as: 

a) creating a blockchain<br> 
b) verifying a chain<br>
c) forking<br> 
d) comparing chains

In [1]:
import copy # fork a chain
import datetime # get real time for timestamps
import hashlib # hash

## Define classes

In [4]:
class MinimalChain():
    
    #Initialization of chain - First block using genesis function
    def __init__(self):
        self.blocks = [self.get_genesis_block()]
        
    #Check equality of chains
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False
    
    #Geneis - Create first block
    def get_genesis_block(self): 
        return MinimalBlock(0, 
                            datetime.datetime.utcnow(), 
                            'Genesis', 
                            'arbitrary')
    
    # Adding a new block to the chain
    def add_block(self, data):
        self.blocks.append(MinimalBlock(len(self.blocks), 
                                        datetime.datetime.utcnow(), 
                                        data, 
                                        self.blocks[len(self.blocks)-1].hash))
    
    # Size excluding genesis block
    def get_chain_size(self): 
        return len(self.blocks)-1
    
    # Verification of a block's validity
    def verify(self, verbose=True): 
        flag = True
        for i in range(1,len(self.blocks)):
            if not self.blocks[i].verify(): # assume Genesis block integrity
                flag = False
                if verbose:
                    print(f'Wrong data type(s) at block {i}.')
            if self.blocks[i].hash[:2] != '00':
                flag = False
                if verbose:
                    print(f'Wrong block hash {i}, must mine it.')
            if self.blocks[i-1].hash != self.blocks[i].previous_hash:
                flag = False
                if verbose:
                    print(f'Wrong previous hash at block {i}.')
            if self.blocks[i].hash != self.blocks[i].hashing():
                flag = False
                if verbose:
                    print(f'Wrong hash at block {i}.')
            if self.blocks[i-1].timestamp > self.blocks[i].timestamp:
                flag = False
                if verbose:
                    print(f'Backdating at block {i}.')
        return flag
    
    # Cloning the chain
    def fork(self, head='latest'):
        if head in ['latest', 'whole', 'all']:
            # deepcopy since lists are mutable objects
            return copy.deepcopy(self) 
        else:
            c = copy.deepcopy(self)
            c.blocks = c.blocks[0:head+1]
            return c
    
    # Check your status compared to chain2 and update up until the block of conflict
    def get_root(self, chain_2):
        min_chain_size = min(self.get_chain_size(), chain_2.get_chain_size())
        for i in range(1,min_chain_size+1):
            if self.blocks[i] != chain_2.blocks[i]:
                return self.fork(i-1)
        return self.fork(min_chain_size)    

In [10]:
class MinimalBlock():
    # Create new block
    def __init__(self, nonce, timestamp, data, previous_hash):
        self.nonce = nonce 
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        if (self.mine() == False):
            print('Mine failed')
        self.hash = self.hashing()
    
    #Check similarity of blocks
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False
        
    #Hash using all fields of block
    def hashing(self):
        key = hashlib.sha256()
        key.update(str(self.nonce).encode('utf-8'))
        key.update(str(self.timestamp).encode('utf-8'))
        key.update(str(self.data).encode('utf-8'))
        key.update(str(self.previous_hash).encode('utf-8'))
        return key.hexdigest()
    
    # mine block - find correct nonce for hash to be starting with '00'
    def mine(self):
        self.nonce = 0
        while(self.hashing()[:2] != '00'):
            self.nonce = self.nonce + 1
        return True
    
    # check data types of all info in a block
    def verify(self):  
        instances = [self.nonce, self.timestamp, self.previous_hash, self.hash]
        types = [int, datetime.datetime, str, str]
        if sum(map(lambda inst_, type_: isinstance(inst_, type_), instances, types)) == len(instances):
            return True
        else:
            return False

## Testing

In [11]:
c = MinimalChain() # Start a chain
for i in range(1,20+1):
    c.add_block(f'This is block {i} of my first chain.')

In [12]:
print(c.blocks[3].timestamp)
print(c.blocks[7].data)
print(c.blocks[9].hash)

2020-03-08 18:04:02.200624
This is block 7 of my first chain.
001be6e206e16d27711ffb539b1720ab33949d3bf5217dcbd648c702537e67b9


In [14]:
print(c.get_chain_size())
print(c.verify())

20
True


In [15]:
c_forked = c.fork('latest')
print(c == c_forked) #same content?
print(c is c_forked) #same object?

True
False


In [16]:
c_forked.add_block('New block for forked chain!')
print(c.get_chain_size(), c_forked.get_chain_size())

20 21


## Conflict Testing

Change nonce of a block

In [17]:
c_forked = c.fork('latest')
c_forked.blocks[9].nonce = -9
c_forked.verify()

Wrong hash at block 9.


False

Try to fix it using mine()

In [18]:
c_forked.blocks[9].mine()
c_forked.verify()

True

Change timestamp manually

In [19]:
c_forked = c.fork('latest')
c_forked.blocks[16].timestamp = datetime.datetime(2000, 1, 1, 0, 0, 0, 0)
c_forked.verify()

Wrong hash at block 16.
Backdating at block 16.


False

Change previous_hash of block   

In [20]:
c_forked = c.fork('latest')
c_forked.blocks[5].previous_hash = c_forked.blocks[1].hash
c_forked.verify()

Wrong previous hash at block 5.
Wrong hash at block 5.


False